In [18]:
from typing import Literal
modo_operazione = Literal['append', 'replace']

anno = '2024'
modo: modo_operazione = 'append'    
# in caso di aggiornamento utilizzare 'append' per inserire in db solo l'anno selezionato, 
# in caso contrario utilizzare 'replace' per ricreare da capo l'intero db ad esclusione delle colonne in colonne_da_eliminare

# Parametri DB
hostname = '185.2.168.125'
username = 'enricoma_user'
password = '932197Silvestr_'
database_name = 'enricoma_lavoro'

In [19]:
import pandas as pd
from sqlalchemy import create_engine, Float, VARCHAR, Text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

pd.set_option("display.max_columns", None)

dtype_mapping_tariffe = {
    'numero_s_vdt':                         VARCHAR(15),
    'edizione_tariffa':                     VARCHAR(10),
    'tariffa':                              VARCHAR(5),
    'gruppo':                               VARCHAR(7),
    'descrizione_estesa_per_gruppo_2024':   Text,
    'numero_vdt':                           VARCHAR(12),
    'descrizione_estesa_per_voce':          Text,
    'titolo':                               Text,
    'stato':                                VARCHAR(4),
    'um':                                   VARCHAR(5),
    'applicazione':                         VARCHAR(1),
    'ean11_uan_2022':                       VARCHAR(5),
    'cella_di_controllo':                   VARCHAR(5),
    'inserimento':                          VARCHAR(3),
    'eliminazioni':                         VARCHAR(4),
    'var_prezzo':                           VARCHAR(3),
    'perc_var':                             Float,
    'prezzo_unitario':                      Float,
    'perc_mo':                              Float
}
dtype_mapping_tariffe[f'descrizione_{anno}'] = Text
dtype_mapping_tariffe[f'testo_esteso_{anno}'] = Text

colonne_da_eliminare = [
    "ean11_uan_2022",
    "ed_1992", 
    "ed_2008", 
    "ed_2009", 
    "ed_2010", 
    "ed_2011", 
    "ed_2012", 
    "ed_2013", 
    "ed_2014", 
    "ed_2015", 
    "ed_2016", 
    "ed_2017", 
    "ed_2018", 
    "ed_2019",
    'ed_2020',
    'ed_2021', 
    'ed_2022', 
    'ed_2022_ec', 
    'ed_2022_04', 
    'ed_2022_05',
    'ed_2022_11',
    "perc_mo_2018", 
    "perc_mo_2019",
    'perc_mo_2020',
    'perc_mo_2021', 
    'perc_mo_2022', 
    'perc_mo_2022_ec', 
    'perc_mo_2022_04',
    'perc_mo_2022_05', 
    'perc_mo_2022_11'    
]


In [20]:
def inserimento_in_db(df, df_macep, if_exist: Literal['append', 'replace']):
    # Primo melt per 'ed_1992' a 'ed_2024'
    df_melted_prezzi = pd.melt(df, id_vars=[col for col in df.columns if not col.startswith('ed_') and not col.startswith('perc_mo_')],
                            value_vars=[col for col in df.columns if col.startswith('ed_')],
                            var_name='edizione_tariffa', value_name='prezzo_unitario')
    df_melted_prezzi.set_index(['numero_s_vdt', 'edizione_tariffa'], drop=True, inplace=True)

    # Secondo melt per 'perc_mo_2018' a 'perc_mo_2024'
    df_melted_perc = pd.melt(df, id_vars=['numero_s_vdt'],
                            value_vars=[col for col in df.columns if col.startswith('perc_mo_')],
                            var_name='edizione_tariffa_perc', value_name='perc_mo')


    # Adattare il nome delle 'edizioni' per fare il match
    df_melted_perc['edizione_tariffa'] = df_melted_perc['edizione_tariffa_perc'].str.replace('perc_mo_', 'ed_')
    df_melted_perc.drop(columns=['edizione_tariffa_perc'], inplace=True)
    df_melted_perc.set_index(['numero_s_vdt', 'edizione_tariffa'], drop=True, inplace=True)
      
    df_final = pd.merge(df_melted_prezzi, df_melted_perc, left_index=True, right_index=True, how='inner')
    df_final['prezzo_unitario'] = df_final['prezzo_unitario'].round(2)
    
    if if_exist == 'append':
        df_final = df_final.loc[df_final.index.get_level_values(1) == f'ed_{anno}']
        
    engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}')
    Session = sessionmaker(bind=engine)
    session = Session()
    try:
        df_final.to_sql('tariffe', con=engine, if_exists=if_exist, index=True, index_label = ['numero_s_vdt', 'edizione_tariffa'], method='multi', dtype=dtype_mapping_tariffe)
    except SQLAlchemyError as e:
        print(f"Si è verificato un errore: {e}")
        session.rollback()
    finally:
        engine.dispose()
        
    # Primo melt per 'ed_1992' a 'ed_2024'
    df_macep_final = pd.melt(df_macep, id_vars=[col for col in df_macep.columns if not col.startswith('ed_') and not col.startswith('perc_mo_')],
                            value_vars=[col for col in df_macep.columns if col.startswith('ed_')],
                            var_name='edizione_tariffa', value_name='prezzo_unitario')
    df_macep_final.set_index(['numero_s_vdt', 'edizione_tariffa'], drop=True, inplace=True)
    df_macep_final.insert(len(df_macep_final.columns), 'perc_mo', 0)
    df_macep_final['prezzo_unitario'] = df_macep_final['prezzo_unitario'].round(2)

    if if_exist == 'append':
        df_macep_final = df_macep_final.loc[df_macep_final.index.get_level_values(1) == f'ed_{anno}']
        
    engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}')
    Session = sessionmaker(bind=engine)
    session = Session()
    try:
        df_macep_final.to_sql('tariffe', con=engine, if_exists='append', index=True, index_label = ['numero_s_vdt', 'edizione_tariffa'], method='multi', dtype=dtype_mapping_tariffe)
    except SQLAlchemyError as e:
        print(f"Si è verificato un errore: {e}")
        session.rollback()
    finally:
        engine.dispose()
    
    return df_final, df_macep_final


In [21]:
df = pd.read_excel(f"DB_tariffe_{anno}.xlsx", sheet_name=f"DB_{anno}")
col_descr = f"Descrizione Estesa per Voce {anno}"
col_titolo = f"Titolo {anno}"
df = df.rename(columns={
    col_descr: "Descrizione Estesa per Voce",
    col_titolo: "Titolo",
    "Testo esteso 2023": "Testo esteso",
    "UdM": "um",
    "EAN11/UAN 2022": "ean11_uan_2022",
    "1992 / 1993": "1992",
    "2022\n(SG=13,5%)": "2022",
    "2022_EC\n(SG=13,5%)": "2022_EC",
    "2022_04_08\n(SG=13,5%)": "2022_04",
    "2022_05 \n(SG=13,5%)": "2022_05",
    "2022_11_EC\n(SG=13,5%)": "2022_11",
    "2023\n(SG=15%)": "2023",
    "2023_EC\n(SG=15%)": "2023_EC",
    "2024\n(SG=15%)": "2024",
    "%MO 2022_04_08": "%MO 2022_04",
    "%MO 2022_11_EC": "%MO 2022_11"
    })

df.columns = [str(col).replace('%', 'perc_') for col in df.columns]   
df.columns = [str(col.strip()).replace(' ', '_') for col in df.columns]   
df.columns = [str(col).replace('.', '_') for col in df.columns] 
nuovi_nomi_colonne = {col: f'ed_{col}' if col[0].isdigit() else col for col in df.columns}
df.rename(columns=nuovi_nomi_colonne, inplace=True)
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('string')

df.columns = [col.lower() for col in df.columns]
df.drop(colonne_da_eliminare, inplace=True, axis=1)
df = df.rename(columns={
    'descrizione_estesa_per_gruppo_2024': 'descrizione_estesa_per_gruppo', 
    'descrizione_2024': 'descrizione',
    'testo_esteso_2024': 'testo_esteso'
})


In [22]:
df_macep = pd.read_excel(f"DB_tariffe_{anno}.xlsx", sheet_name=f"MaCeP_{anno}")
df_macep = df_macep.rename(columns={
    "2022 (SG=13,5%)": "2022",
    "2022_04_08 (SG=13,5%)": "2022_04",
    "2022_05 (SG=13,5%)": "2022_05",
    "2023\n(K=10% SG=15%)": "2023",
    "2024\n(K=10% SG=15%)": "2024",
    })
df_macep.columns = [str(col).replace('%', 'perc_') for col in df_macep.columns]   
df_macep.columns = [str(col.strip()).replace(' ', '_') for col in df_macep.columns]   
df_macep.columns = [str(col).replace('.', '_') for col in df_macep.columns]  
nuovi_nomi_colonne = {col: f'ed_{col}' if col[0].isdigit() else col for col in df_macep.columns}
df_macep.rename(columns=nuovi_nomi_colonne, inplace=True)
for col in df_macep.columns:
    if df_macep[col].dtype == 'object':
        df_macep[col] = df_macep[col].astype('string')
df_macep.columns = [col.lower() for col in df_macep.columns]
df_macep.drop(colonne_da_eliminare, inplace=True, axis=1, errors='ignore')
df_macep.rename(columns={'codice_materiale': 'numero_s_vdt'}, inplace=True)
df_macep.insert(1, 'tariffa', 'MACEP')
df_macep.insert(2, 'gruppo', 'MACEP')
df_macep.insert(3, 'descrizione_estesa_per_gruppo', df_macep['descrizione'])
df_macep.insert(4, 'numero_vdt', df_macep['numero_s_vdt'])
df_macep.insert(5, 'descrizione_estesa_per_voce', df_macep['descrizione'])
df_macep.insert(7, 'titolo', df_macep['descrizione'])
df_macep.insert(8, 'testo_esteso', df_macep['descrizione'])
df_macep.insert(9, 'stato', 'RIL')
df_macep.insert(11, 'applicazione', 'X')

In [23]:
df, df_macep = inserimento_in_db(df, df_macep, if_exist=modo)
print('fine')

fine
